## MVPA Hyperalignment

In [ ]:
"""
@author: Donggeun Kim
@affiliation: NYSPI, Columbia University
@date: Oct 2018 - Oct 2020
@overview: Generates Cross-Subject-Validation Fold datasets using ANOVA and MVPA hyperalignment.
@input: Brain Tissue Mask (mask.nii), subject-specific minimally processed fMRI scans.
@output: LOOCV Train/Test folds of 300-length hyperaligned voxels as numpy arrays. LOOCV Train/Test labels.
"""

In [ ]:
import mvpa2
import glob
import os
from mvpa2.suite import *

In [ ]:
datapath=r'D:\dataprocess\shared\MDD\AFNI\MRIunit_More_Than_One_Run'
maskname = [r'D:\dataprocess\shared\MDD\AFNI\MRIunit_More_Than_One_Run\E13260\prep_firstlevel\firstLevel_fmri_trigger_scan_9__007\mask.img']

attributename=[]
filename=[]
#maskname=[]
subjectID=[]
for root, dirs, files in os.walk(datapath):
    if root.find('E13905')<0:
        w_files = glob.glob(os.path.join(root,'wrast*_attributes.txt'))
        f_files = glob.glob(os.path.join(root,'f*_attributes.txt'))
        subjectID.append(dirs)
        for f in w_files:
            filename.append(os.path.abspath(f)[:-15]+'.nii')
        for f in f_files:
            attributename.append(os.path.abspath(f))
    #        maskname.append(os.path.abspath(f)[:-14]+'mask.nii')
subjectID = subjectID[0]
subjectID.remove('E13905')

In [ ]:
subjectID

['E13260',
 'E13341',
 'E13495',
 'E13544',
 'E13550',
 'E13571',
 'E14081',
 'E14133',
 'E14146',
 'E14163',
 'E14178']

In [ ]:
fds = []
for subject in subjectID:
    attribute_subject = [attribute_subject for attribute_subject in attributename if attribute_subject.find(subject)>0]
    file_subject = [file_subject for file_subject in filename if file_subject.find(subject)>0]
#    mask_subject = [mask_subject for mask_subject in maskname if mask_subject.find(subject)>0]
    fds_subject = []
    for i in range(len(attribute_subject)):
        bold_fname = file_subject[i]
#        mask_fname = mask_subject[-1]
        mask_fname=maskname[-1]
        attr_fname = attribute_subject[i]
        attr = SampleAttributes(attr_fname)
        fds_subject += [fmri_dataset(samples=bold_fname,
                    targets=attr.targets, chunks=attr.chunks,
                    mask=mask_fname)]
    fds += [mvpa2.base.dataset.vstack(fds_subject,a=0)]


#### Within Subject
http://www.pymvpa.org/examples/hyperalignment.html

In [ ]:
ds_all = fds
_ = [zscore(ds) for ds in ds_all]
for i, sd in enumerate(ds_all):
    sd.sa['subject'] = np.repeat(i, len(sd))
nsubjs = len(ds_all) # number of subjects
ncats = len(ds_all[0].UT) # number of labels
nruns = len(ds_all[0].UC) # number of runs
print "%d subjects" % len(ds_all)
print "Per-subject dataset: %i samples with %i features" % ds_all[0].shape
print "Stimulus categories: %s" % ', '.join(ds_all[0].UT)
# use same classifier
clf = LinearCSVMC()

# feature selection helpers
nf = 300
fselector = FixedNElementTailSelector(nf, tail='upper',
                                      mode='select', sort=False)
sbfs = SensitivityBasedFeatureSelection(OneWayAnova(), fselector,
                                        enable_ca=['sensitivities'])
# create classifier with automatic feature selection
fsclf = FeatureSelectionClassifier(clf, sbfs)
print "Performing classification analyses..."
print "within-subject..."
wsc_start_time = time.time()
cv = CrossValidation(fsclf,
                     NFoldPartitioner(attr='chunks'),
                     errorfx=mean_match_accuracy)
# store results in a sequence
wsc_results = [cv(sd) for sd in ds_all]
wsc_results = vstack(wsc_results)
print " done in %.1f seconds" % (time.time() - wsc_start_time,)

11 subjects
Per-subject dataset: 400 samples with 47828 features
Stimulus categories: 0, 1, 2
Performing classification analyses...
within-subject...
 done in 9.2 seconds


In [ ]:
print "between-subject (hyperaligned)..."
hyper_start_time = time.time()
bsc_hyper_results = []
# same cross-validation over subjects as before
cv = CrossValidation(clf, NFoldPartitioner(attr='subject'),
                     errorfx=mean_match_accuracy)

# leave-one-run-out for hyperalignment training
for test_run in range(nruns):
    # split in training and testing set
    ds_train = [sd[sd.sa.chunks != test_run, :] for sd in ds_all]
    ds_test = [sd[sd.sa.chunks == test_run, :] for sd in ds_all]

    # manual feature selection for every individual dataset in the list
    anova = OneWayAnova()
    fscores = [anova(sd) for sd in ds_train]
    featsels = [StaticFeatureSelection(fselector(fscore)) for fscore in fscores]
    ds_train_fs = [fs.forward(sd) for fs, sd in zip(featsels, ds_train)]


    # Perform hyperalignment on the training data with default parameters.
    # Computing hyperalignment parameters is as simple as calling the
    # hyperalignment object with a list of datasets. All datasets must have the
    # same number of samples and time-locked responses are assumed.
    # Hyperalignment returns a list of mappers corresponding to subjects in the
    # same order as the list of datasets we passed in.


    hyper = Hyperalignment()
    hypmaps = hyper(ds_train_fs)

    # Applying hyperalignment parameters is similar to applying any mapper in
    # PyMVPA. We start by selecting the voxels that we used to derive the
    # hyperalignment parameters. And then apply the hyperalignment parameters
    # by running the test dataset through the forward() function of the mapper.

    ds_test_fs = [fs.forward(sd) for fs, sd in zip(featsels, ds_test)]
    ds_hyper = [h.forward(sd) for h, sd in zip(hypmaps, ds_test_fs)]

    # Now, we have a list of datasets with feature correspondence in a common
    # space derived from the training data. Just as in the between-subject
    # analyses of anatomically aligned data we can stack them all up and run the
    # crossvalidation analysis.

    ds_hyper = vstack(ds_hyper)
    # zscore each subject individually after transformation for optimal
    # performance
    zscore(ds_hyper, chunks_attr='subject')
    res_cv = cv(ds_hyper)
    bsc_hyper_results.append(res_cv)

bsc_hyper_results = hstack(bsc_hyper_results)
print "done in %.1f seconds" % (time.time() - hyper_start_time,)

between-subject (hyperaligned)...
done in 43.3 seconds


In [ ]:
#Experiments found nf=300 to be an appropriate value for the number of selected voxels.

nf=300
hyper_start_time = time.time()
bsc_hyper_results = []
# same cross-validation over subjects as before
cv = CrossValidation(clf, NFoldPartitioner(attr='subject'),
                     errorfx=mean_match_accuracy)
anova = OneWayAnova()
fscores = [anova(sd) for sd in ds_all]
featsels = [StaticFeatureSelection(fselector(fscore)) for fscore in fscores]
ds_fs = [fs.forward(sd) for fs, sd in zip(featsels, ds_all)]

hyper = Hyperalignment()
hypmaps = hyper(ds_fs)



In [ ]:
ds_hyper = [h.forward(sd) for h, sd in zip(hypmaps, ds_fs)]
ds_hyper = vstack(ds_hyper)

In [ ]:
ds_hyper.targets.astype(int)

array([0, 0, 0, ..., 2, 2, 2])

In [ ]:
np.savetxt("X_hyp_v2.csv", ds_hyper.samples, delimiter=",")
np.savetxt("Y_hyp_v2.csv", ds_hyper.targets.astype(int), delimiter=",")
